# Dataanalyse for alle produkter
Med udgangspunkt i en sortering af de vigtigste produkter i prioriteret rækkefølge

In [2]:
import pandas as pd
import openpyxl

vareposter = pd.read_excel('Vareposter Alfotech.xlsx', header=0)
unikke_varenumre = vareposter['Varenr.'].unique()
unikke_varenumre = pd.DataFrame(unikke_varenumre, columns=['Varenr.'])

In [ ]:
display(unikke_varenumre)
unikke_varenumre.to_excel('unikke_varenumre.xlsx', index=False)

,Varenr.
0,010153492910
1,0278021003
2,04211S1008
3,0418000250
4,0290029100
...,...
8070,0301029150
8071,250203803000101
8072,2518051080003NB0
8073,25183641000B0B0


In [4]:
vareposter_salg = vareposter[vareposter['Posttype'].isin(['Salg', 'Montageforbrug'])]
vareposter_køb = vareposter[vareposter['Posttype'] == 'Køb']

In [11]:
vareposter_salg.to_excel('vareposter_salg.xlsx', index=False)
vareposter_køb.to_excel('vareposter_køb.xlsx', index=False)

In [17]:
# Calculate total 'Antal' sold per product and supplier, sorted by most sold
antal_per_varenr_leverandor = (
    vareposter_salg.groupby(['Varenr.', 'Leverandørnr.'])['Antal']
    .sum()
    .reset_index()
    .sort_values('Antal', ascending=True)  # Most sold products at the top (negative values)
)

In [24]:
# Calculate total 'Antal' purchased per product and supplier, sorted by most sold
antal_per_varenr_leverandor_purchased = (
    vareposter_køb.groupby(['Varenr.', 'Leverandørnr.'])['Antal']
    .sum()
    .reset_index()
    .sort_values('Antal', ascending=False)  # Most purchased products at the top (negative values)
)

In [ ]:
display(antal_per_varenr_leverandor)
antal_per_varenr_leverandor.to_excel('antal_per_varenr_leverandor.xlsx', index=False)

In [ ]:
display(antal_per_varenr_leverandor_purchased)
antal_per_varenr_leverandor_purchased.to_excel('antal_per_varenr_leverandor_purchased.xlsx', index=False)

In [23]:
antal_uden_leverandør = (
    vareposter_salg.loc[vareposter_salg["Leverandørnr."].isnull()]
        .groupby("Varenr.", as_index=False)["Antal"]
        .sum()
        .sort_values("Antal", ascending=True)
        .reset_index(drop=True)
)

In [24]:
antal_uden_leverandør.to_excel('antal_uden_leverandør.xlsx', index=False)

In [13]:
display(antal_uden_leverandør)

,Varenr.,Antal
96,0360231110G,-700.0
308,0469009100,-570.0
342,0597200020,-407.0
298,0466101014,-332.0
35,0290009110,-288.0
...,...,...
300,0467011018,0.0
301,0467031018,0.0
594,250105102000101,2.0
3004,253051009500404G,2.0


In [59]:
import pandas as pd

def analyze_shared_varenr_sales(path_to_excel: str, sheet_name=None, output_path="salg_med_flere_leverandoerer.xlsx"):
    # Load
    df = pd.read_excel(path_to_excel, sheet_name=sheet_name)

    # Basic sanity / required columns
    expected = {"Varenr.", "Antal", "Leverandørnr."}
    missing = expected - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # Normalize varenr. to avoid hidden duplicates
    df["Varenr."] = df["Varenr."].astype(str).str.strip().str.upper()
    df["Leverandørnr."] = df["Leverandørnr."].astype(str).str.strip()

    # Coerce Antal to numeric, drop invalid
    df["Antal"] = pd.to_numeric(df["Antal"], errors="coerce")
    df = df.dropna(subset=["Antal"])

    # Keep only actual sales (negative quantities)
    df_sales = df[df["Antal"] < 0].copy()

    # Count unique suppliers per varenr.
    supplier_counts = (
        df_sales.groupby("Varenr.")["Leverandørnr."]
        .nunique()
        .reset_index(name="Antal leverandører")
    )

    # Filter to varenr. sold by more than one supplier
    shared_varenr = supplier_counts[supplier_counts["Antal leverandører"] > 1]["Varenr."]

    if shared_varenr.empty:
        print("No Varenr. found with more than one supplier in sales.")
        return pd.DataFrame()  # nothing to do

    # Restrict to those varenr.
    df_filtered = df_sales[df_sales["Varenr."].isin(shared_varenr)].copy()

    # Sum sold per (Varenr., Leverandørnr.)
    summary = (
        df_filtered
        .groupby(["Varenr.", "Leverandørnr."])["Antal"]
        .sum()
        .reset_index()
        .rename(columns={"Antal": "Total_Antal"})  # negative
    )

    # Convert to positive sold quantity
    summary["Antal_solgt"] = -summary["Total_Antal"]

    # Merge back supplier counts
    summary = summary.merge(supplier_counts, on="Varenr.", how="left")

    # Compute share of that varenr.'s total sold for each supplier
    total_per_varenr = summary.groupby("Varenr.")["Antal_solgt"].transform("sum")
    summary["Andel_pct"] = (summary["Antal_solgt"] / total_per_varenr * 100).round(1)

    # Sort for readability
    summary = summary.sort_values(["Varenr.", "Antal_solgt"], ascending=[True, False])

    # Save out
    summary.to_excel(output_path, index=False)
    print(f"Saved result to {output_path}")

    return summary


NameError: name 'summary' is not defined

In [ ]:
# Show all the rows where Kildenr. is not the same as Leverandørnr.
not_same_Kildenr_leverandørnr = vareposter_køb[vareposter_køb['Kildenr.'] != vareposter_køb['Leverandørnr.']]
display(not_same_Kildenr_leverandørnr)

,Bogføringsdato,Posttype,Bilagstype,Bilagsnr.,Varenr.,Beskrivelse,Lokationskode,Antal,Faktureret antal,Restantal,Salgsbeløb (faktisk),Kostbeløb (faktisk),Kostbeløb (ikke-lager),Åben,Ordretype,Løbenr.,Kildetype,Kildenr.,Leverandørnr.
669,2025-04-16,Køb,Købsleverance,20924634,0100029080,NaN,LAGER,40.0,40.0,0.0,0.0,348.00,0.0,False,NaN,226879,Kreditor,87944000,86892211
670,2025-04-16,Køb,Købsleverance,20924633,0101001016,NaN,LAGER,5.0,5.0,0.0,0.0,1476.20,0.0,False,NaN,226878,Kreditor,87944000,86892211
961,2025-04-14,Køb,Købsleverance,20924596,NaN,NaN,LAGER,1.0,1.0,0.0,0.0,168.03,0.0,False,NaN,226538,Kreditor,86892211,NaN
1072,2025-04-10,Køb,Købsleverance,20924722,2069309025,Elektropolering af 0100029080 -adapter,LAGER,6.0,6.0,6.0,0.0,550.00,0.0,True,NaN,228669,Kreditor,86299911,87371811
1138,2025-04-10,Køb,Købsleverance,20924545,0101001016,NaN,LAGER,2.0,2.0,0.0,0.0,492.06,0.0,False,NaN,226366,Kreditor,87944000,86892211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60193,2023-05-02,Køb,Købsleverance,20917300,0101000510,NaN,LAGER,20.0,20.0,0.0,0.0,938.80,0.0,False,NaN,156009,Kreditor,87944000,86892211
60205,2023-05-02,Køb,Købsleverance,20917298,0101321000,NaN,LAGER,9.0,9.0,0.0,0.0,670.23,0.0,False,NaN,155997,Kreditor,87944000,86892211
60397,2023-05-01,Køb,Købsleverance,20917265,0101020510,NaN,LAGER,10.0,10.0,0.0,0.0,461.70,0.0,False,NaN,155761,Kreditor,87944000,86892211
60398,2023-05-01,Køb,Købsleverance,20917265,0100009100,NaN,LAGER,10.0,10.0,0.0,0.0,140.00,0.0,False,NaN,155760,Kreditor,87944000,86892211


In [ ]:
not_same_Kildenr_leverandørnr.to_excel('not_same_kildenr_leverandørnr.xlsx', index=False)

In [ ]:
# Describe the dataframe
not_same_Kildenr_leverandørnr.describe()
not_same_Kildenr_leverandørnr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 872 entries, 669 to 60401
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Bogføringsdato          872 non-null    datetime64[ns]
 1   Posttype                872 non-null    object        
 2   Bilagstype              871 non-null    object        
 3   Bilagsnr.               872 non-null    object        
 4   Varenr.                 871 non-null    object        
 5   Beskrivelse             224 non-null    object        
 6   Lokationskode           844 non-null    object        
 7   Antal                   872 non-null    float64       
 8   Faktureret antal        872 non-null    float64       
 9   Restantal               872 non-null    float64       
 10  Salgsbeløb (faktisk)    872 non-null    float64       
 11  Kostbeløb (faktisk)     872 non-null    float64       
 12  Kostbeløb (ikke-lager)  872 non-null    float64    

In [73]:
# make sure Antal is numeric (coerce non-numeric to NaN then zero)
not_same_kildenr_leverandørnr['Antal'] = pd.to_numeric(
    not_same_kildenr_leverandørnr['Antal'], errors='coerce'
).fillna(0)

# group by Kildenr and sum Antal
sum_by_kildenr = (
    not_same_kildenr_leverandørnr
    .groupby('Kildenr.', dropna=False, observed=False)['Antal']
    .sum()
    .reset_index(name='Total_Antal')
    .sort_values('Total_Antal', ascending=False)
)


C:\Users\sgj\AppData\Local\Temp\ipykernel_24016\3641793399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_same_kildenr_leverandørnr['Antal'] = pd.to_numeric(


In [74]:
display(sum_by_kildenr)

,Kildenr.,Total_Antal
23,87944000,5820.0
20,87371811,3225.0
3,44576200,515.0
24,BE36801190,502.0
40,NL736430288,375.0
38,NL472822,350.0
19,86892211,317.0
9,64632423,277.0
17,86299911,266.0
27,DE406394338,264.0


In [84]:
# make a working copy and ensure numeric
df = not_same_kildenr_leverandørnr.copy()
df['Antal'] = pd.to_numeric(df['Antal'], errors='coerce').fillna(0)

# optional: normalize keys to avoid silent mismatches
df['Kildenr.'] = df['Kildenr.'].astype(str).str.strip()
df['Varenr.'] = df['Varenr.'].astype(str).str.strip()

# total per Kildenr+Varenr (if there are duplicate rows you want collapsed)
df['Antal_per_Varenr'] = df.groupby(['Kildenr.', 'Varenr.'])['Antal'].transform('sum')

# collapse to one row per Kildenr+Varenr with both aggregates
summary = (
    df
    .drop_duplicates(subset=['Kildenr.', 'Varenr.'])
    .loc[:, ['Kildenr.', 'Varenr.', 'Antal_per_Varenr']]
    .sort_values(['Antal_per_Varenr'], ascending=False)
    .reset_index(drop=True)
)
display(summary)

,Kildenr.,Varenr.,Antal_per_Varenr
0,87371811,0491001004,363.0
1,87371811,0499025037,353.0
2,87944000,0100039100,350.0
3,NL472822,0450000021,300.0
4,87944000,0101000250,290.0
...,...,...,...
406,87944000,0101090651,1.0
407,86892211,nan,1.0
408,44600302,1400187537,0.0
409,44921088,0403750400,0.0


In [88]:
summary.to_excel('kildenr_køb_data.xlsx', index=False)

In [2]:
vareposter.head()

,Bogføringsdato,Posttype,Bilagstype,Bilagsnr.,Varenr.,Beskrivelse,Lokationskode,Antal,Faktureret antal,Restantal,Salgsbeløb (faktisk),Kostbeløb (faktisk),Kostbeløb (ikke-lager),Åben,Ordretype,Løbenr.,Kildetype,Kildenr.,Leverandørnr.
0,2025-04-30,Køb,Købsleverance,20924732,010153492910,NaN,LAGER,4.0,4.0,0.0,0.00,434.16,0.0,False,NaN,228867,Kreditor,86243311,86243311
1,2025-04-30,Salg,Salgsleverance,112152,0278021003,"Push-in, male connector, AISI 316,",LAGER,-4.0,-4.0,0.0,595.75,-204.44,0.0,False,NaN,228646,Debitor,GB35316289530,IT2661006
2,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.83,-99.92,0.0,False,NaN,228645,Debitor,GB35316289530,IT307401249
3,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.84,-99.92,0.0,False,NaN,228644,Debitor,GB35316289530,IT307401249
4,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.84,-99.92,0.0,False,NaN,228643,Debitor,GB35316289530,IT307401249


In [9]:
import pandas as pd

# === Configurable names ===
key_col = "Nummer"  # change if the item number column is named differently
wanted_columns = [
    "Beskrivelse",
    "Beskrivelse 2",
    "Beskrivelse 3",
    "Basisenhed",
    "Kostpris",
    "Enhedspris",
]

# === Load the list of unique item numbers ===
unikke = pd.read_excel("unikke_varenumre.xlsx", usecols=[key_col])
unikke[key_col] = unikke[key_col].astype(str).str.strip()

# === Load Varer.xlsx with only the needed columns ===
required_cols = [key_col] + wanted_columns
varer = pd.read_excel("Varer.xlsx", usecols=required_cols)
varer[key_col] = varer[key_col].astype(str).str.strip()

# === Optional: check for missing expected columns early ===
missing = [c for c in required_cols if c not in varer.columns]
if missing:
    raise ValueError(f"These expected columns are missing from Varer.xlsx: {missing}")

# === Perform the match ===
matched = unikke.merge(varer, on=key_col, how="inner")

# === Report unmatched item numbers ===
not_found = unikke.loc[~unikke[key_col].isin(varer[key_col])].drop_duplicates()
if not not_found.empty:
    print(f"{len(not_found)} item numbers from unikke_varenumre.xlsx had no match in Varer.xlsx. Sample:\n", not_found.head())

# === Save result ===
matched.to_excel("matched_varenumre.xlsx", index=False)
print(f"Matched result written to 'matched_varenumre.xlsx'. Rows: {len(matched)}")


60 item numbers from unikke_varenumre.xlsx had no match in Varer.xlsx. Sample:
               Nummer
432  25124120200B0E0
451    2507951030033
466              nan
755     013125058544
839   250560751000BB
Matched result written to 'matched_varenumre.xlsx'. Rows: 8015


In [28]:
import pandas as pd

# === Configurable names ===
key_col = "Nummer"  # item number column
wanted_columns = [
    "Beskrivelse",
    "Beskrivelse 2",
    "Beskrivelse 3",
    "Basisenhed",
    "Kostpris",
    "Enhedspris",
]

# === Load the list of item numbers with their Antal ===
unikke = pd.read_excel("antal_uden_leverandør.xlsx", usecols=[key_col, "Antal"])
unikke[key_col] = unikke[key_col].astype(str).str.strip()

# Ensure Antal is numeric (if it isn't already)
unikke["Antal"] = pd.to_numeric(unikke["Antal"], errors="coerce")

# === Load Varer.xlsx with only the needed columns ===
required_cols = [key_col] + wanted_columns
varer = pd.read_excel("Varer.xlsx", usecols=required_cols)
varer[key_col] = varer[key_col].astype(str).str.strip()

# === Check for missing expected columns early ===
missing = [c for c in required_cols if c not in varer.columns]
if missing:
    raise ValueError(f"These expected columns are missing from Varer.xlsx: {missing}")

# === Perform the match ===
matched = unikke.merge(varer, on=key_col, how="inner")

# === Report unmatched item numbers ===
not_found = unikke.loc[~unikke[key_col].isin(varer[key_col])].drop_duplicates()
if not not_found.empty:
    print(
        f"{len(not_found)} item numbers from antal_uden_leverandør.xlsx had no match in Varer.xlsx. Sample:\n",
        not_found.head(),
    )

# === Save result ===
matched.to_excel("matched_varenumre_uden_lev.xlsx", index=False)
print(f"Matched result written to 'matched_varenumre_uden_lev.xlsx'. Rows: {len(matched)}")


49 item numbers from antal_uden_leverandør.xlsx had no match in Varer.xlsx. Sample:
               Nummer  Antal
83    2636151007599A  -30.0
143  25346206000B0B0  -20.0
167    2528925012044  -18.0
172    2528925010044  -18.0
178    2523013H147NN  -17.0
Matched result written to 'matched_varenumre_uden_lev.xlsx'. Rows: 3443


In [29]:
import pandas as pd
import numpy as np

# Load datasets
salg = pd.read_excel("salg_per_leverandør.xlsx")
matched = pd.read_excel("matched_varenumre_uden_lev.xlsx")  # expects columns "Varenr." and "Antal"

# Ensure numeric and positive Antal
matched["Antal_pos"] = matched["Antal"].astype(float).abs()

# Prepare the descending sorted unique Solgt_per_leverandør values
solgt_vals = (
    salg["Solgt_per_leverandør"]
        .dropna()
        .sort_values(ascending=False)
        .unique()
)
# If not enough values to compare, skip insertions gracefully
if len(solgt_vals) < 2:
    print("Not enough distinct Solgt_per_leverandør values to determine 'between' positions.")
    
# Build new rows to insert
new_rows = []
for _, row in matched.iterrows():
    val = row["Antal_pos"]
    # find if val is between any two consecutive descending solgt_vals
    if len(solgt_vals) >= 2:
        # condition inclusive: solgt_vals[i] >= val >= solgt_vals[i+1]
        between_mask = (solgt_vals[:-1] >= val) & (val >= solgt_vals[1:])
        if not between_mask.any():
            # skip if not in between any pair
            continue
    else:
        continue  # can't place if no interval

    # create a blank template row with all columns, then fill required ones
    template = {col: pd.NA for col in salg.columns}
    template["Varenr."] = row["Varenr."]
    template["Solgt uden leverandørnr."] = val
    # give it a sort key so it lands correctly among existing rows
    template["_sort_key"] = val
    new_rows.append(template)

# If no new rows, warn
if not new_rows:
    print("No matched Antal values fell between any two Solgt_per_leverandør values; nothing inserted.")
    combined = salg.copy()
else:
    # Prepare existing data with sort key from Solgt_per_leverandør
    combined = salg.copy()
    combined["_sort_key"] = combined["Solgt_per_leverandør"]

    # Append new rows
    extras = pd.DataFrame.from_records(new_rows, columns=combined.columns)
    combined = pd.concat([combined, extras], ignore_index=True)

    # Sort so inserted rows appear where their Antal would fit
    combined = combined.sort_values("_sort_key", ascending=False, ignore_index=True)

# Drop helper column
if "_sort_key" in combined.columns:
    combined = combined.drop(columns=["_sort_key"])

# Save result
combined.to_excel("salg_per_leverandør_with_inserted_rows.xlsx", index=False)
print(f"Result written to 'salg_per_leverandør_with_inserted_rows.xlsx'. Total rows: {len(combined)}")


C:\Users\sgj\AppData\Local\Temp\ipykernel_17080\3502200088.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([combined, extras], ignore_index=True)


Result written to 'salg_per_leverandør_with_inserted_rows.xlsx'. Total rows: 7903


In [5]:
display(vareposter_salg)

,Bogføringsdato,Posttype,Bilagstype,Bilagsnr.,Varenr.,Beskrivelse,Lokationskode,Antal,Faktureret antal,Restantal,Salgsbeløb (faktisk),Kostbeløb (faktisk),Kostbeløb (ikke-lager),Åben,Ordretype,Løbenr.,Kildetype,Kildenr.,Leverandørnr.
1,2025-04-30,Salg,Salgsleverance,112152,0278021003,"Push-in, male connector, AISI 316,",LAGER,-4.0,-4.0,0.0,595.75,-204.44,0.0,False,NaN,228646,Debitor,GB35316289530,IT2661006
2,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.83,-99.92,0.0,False,NaN,228645,Debitor,GB35316289530,IT307401249
3,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.84,-99.92,0.0,False,NaN,228644,Debitor,GB35316289530,IT307401249
4,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.84,-99.92,0.0,False,NaN,228643,Debitor,GB35316289530,IT307401249
5,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.84,-99.92,0.0,False,NaN,228642,Debitor,GB35316289530,IT307401249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,2023-05-01,Salg,Salgsleverance,98391,0101300320,NaN,LAGER,-2.0,-2.0,0.0,99.00,-37.93,0.0,False,NaN,155766,Debitor,54347222,IN2649750
60394,2023-05-01,Salg,Salgsleverance,98391,0101550500,NaN,LAGER,-2.0,-2.0,0.0,666.00,-288.77,0.0,False,NaN,155765,Debitor,54347222,DE39405920
60395,2023-05-01,Salg,Salgsleverance,98391,0101360500,NaN,LAGER,-4.0,-4.0,0.0,676.80,-225.24,0.0,False,NaN,155764,Debitor,54347222,DE39405920
60396,2023-05-01,Salg,Salgsleverance,98391,0101360400,NaN,LAGER,-6.0,-6.0,0.0,945.00,-313.86,0.0,False,NaN,155763,Debitor,54347222,DE39405920


In [8]:
import pandas as pd

# Paths to the input datasets
sales_path = 'salg_per_leverandør_with_inserted_rows.xlsx'
amount_path = 'vareposter_salg.xlsx'
output_path = 'vareposter_antal_salg.xlsx'

# 1. Load datasets
df_sales = pd.read_excel(sales_path)
df_purchases = pd.read_excel(amount_path)

# 2. Ensure consistent types for merging
df_sales['Varenr.'] = df_sales['Varenr.'].astype(str)
df_purchases['Varenr.'] = df_purchases['Varenr.'].astype(str)

# 3. Deduplicate purchases by de tre nøgler
df_unique_sales = df_purchases.drop_duplicates(subset=['Bogføringsdato', 'Bilagsnr.', 'Varenr.'])

# 4. Tæl unikke salg per Varenr.
sales_counts = (
    df_unique_sales
    .groupby('Varenr.')
    .size()
    .reset_index(name='Antal_salg')
)

# 5. Merge tællinger ind i salgstabellen
df_merged = df_sales.merge(sales_counts, on='Varenr.', how='left')
df_merged['Antal_salg'] = df_merged['Antal_salg'].fillna(0).astype(int)

# 6. Indsæt kolonnen lige efter 'Varenr.'
var_index = df_merged.columns.get_loc('Varenr.')
df_merged.insert(var_index + 1, 'Antal_salg', df_merged.pop('Antal_salg'))

# 7. Gem til ny Excel-fil
df_merged.to_excel(output_path, index=False)

print("Script executed successfully. Output file:", output_path)


Script executed successfully. Output file: vareposter_antal_salg.xlsx


In [12]:
# Ny antal salg med modregning

import pandas as pd

# Sti til input- og outputfiler
sales_path    = 'salg_per_leverandør_with_inserted_rows.xlsx'
amount_path   = 'vareposter_salg.xlsx'
output_path   = 'vareposter_antal_salg_modregning.xlsx'

# 1. Indlæs data
df_sales     = pd.read_excel(sales_path)
df_purchases = pd.read_excel(amount_path)

# 2. Sørg for, at 'Varenr.' er streng for sammenligning
df_sales['Varenr.']     = df_sales['Varenr.'].astype(str)
df_purchases['Varenr.'] = df_purchases['Varenr.'].astype(str)

# 3. Fjern dubletter baseret på de tre nøgler
df_unique = df_purchases.drop_duplicates(subset=['Bogføringsdato', 'Bilagsnr.', 'Varenr.'])

# 4a. Udtræk kun salg (ekskl. returvarekvitteringer)
mask_retour = df_unique['Bilagstype'].str.contains('Salgsreturvarekvittering', na=False)
df_returns   = df_unique[mask_retour]
df_sales_only= df_unique[~mask_retour]

# 4b. Beregn antal unikke salg pr. Varenr.
sales_counts = (
    df_sales_only
    .groupby('Varenr.')
    .size()
    .reset_index(name='Salg')
)

# 4c. Beregn antal unikke returneringer pr. Varenr.
return_counts = (
    df_returns
    .groupby('Varenr.')
    .size()
    .reset_index(name='Retur')
)

# 4d. Slå sammen og beregn nettosalget
counts = (
    sales_counts
    .merge(return_counts, on='Varenr.', how='left')
    .fillna(0)
)
counts['Antal_salg'] = counts['Salg'] - counts['Retur']
counts = counts[['Varenr.', 'Antal_salg']]

# 5. Slå Antal_salg ind i dit oprindelige salgstabel
df_merged = df_sales.merge(counts, on='Varenr.', how='left')
df_merged['Antal_salg'] = df_merged['Antal_salg'].fillna(0).astype(int)

# 6. Indsæt kolonnen straks efter 'Varenr.'
idx = df_merged.columns.get_loc('Varenr.')
df_merged.insert(idx + 1, 'Antal_salg', df_merged.pop('Antal_salg'))

# 7. Gem til ny Excel-fil
df_merged.to_excel(output_path, index=False)

print("Færdig! Fil gemt som:", output_path)


Færdig! Fil gemt som: vareposter_antal_salg_modregning.xlsx


In [15]:
import pandas as pd

# Paths to the input datasets
sales_path = 'vareposter_antal_salg.xlsx'
amount_path = 'vareposter_køb.xlsx'
output_path = 'vareposter_antal_salg_køb.xlsx'

# 1. Load datasets
df_sales = pd.read_excel(sales_path)
df_purchases = pd.read_excel(amount_path)

# 2. Ensure consistent types for merging
df_sales['Varenr.'] = df_sales['Varenr.'].astype(str)
df_purchases['Varenr.'] = df_purchases['Varenr.'].astype(str)

# 3. Deduplicate purchases by de tre nøgler
df_unique_purchases = df_purchases.drop_duplicates(subset=['Bogføringsdato', 'Bilagsnr.', 'Varenr.'])

# 4. Tæl unikke salg per Varenr.
purchase_counts = (
    df_unique_purchases
    .groupby('Varenr.')
    .size()
    .reset_index(name='Antal_køb')
)

# 5. Merge tællinger ind i salgstabellen
df_merged = df_sales.merge(purchase_counts, on='Varenr.', how='left')
df_merged['Antal_køb'] = df_merged['Antal_køb'].fillna(0).astype(int)

# 6. Indsæt kolonnen lige efter 'Varenr.'
var_index = df_merged.columns.get_loc('Antal_salg')
df_merged.insert(var_index + 1, 'Antal_køb', df_merged.pop('Antal_køb'))

# 7. Gem til ny Excel-fil
df_merged.to_excel(output_path, index=False)

print("Script executed successfully. Output file:", output_path)


Script executed successfully. Output file: vareposter_antal_salg_køb.xlsx


In [14]:
# Ny antal køb med modregning

import pandas as pd

# Paths to the input and output files
sales_path    = 'vareposter_antal_salg.xlsx'
purchases_path = 'vareposter_køb.xlsx'
output_path   = 'vareposter_antal_salg_køb_modregning.xlsx'

# 1. Indlæs data
df_sales     = pd.read_excel(sales_path)
df_purchases = pd.read_excel(purchases_path)

# 2. Sørg for, at 'Varenr.' er streng for sammenligning
df_sales['Varenr.']     = df_sales['Varenr.'].astype(str)
df_purchases['Varenr.'] = df_purchases['Varenr.'].astype(str)

# 3. Fjern dubletter baseret på de tre nøgler
df_unique_purchases = df_purchases.drop_duplicates(
    subset=['Bogføringsdato', 'Bilagsnr.', 'Varenr.']
)

# 4a. Adskil køb og returkøbsbilag
mask_retour = df_unique_purchases['Bilagstype']\
    .str.contains('Købsreturvarekvittering', na=False)
df_returns  = df_unique_purchases[mask_retour]
df_buys     = df_unique_purchases[~mask_retour]

# 4b. Beregn antal unikke køb pr. Varenr.
buy_counts = (
    df_buys
    .groupby('Varenr.')
    .size()
    .reset_index(name='Køb')
)

# 4c. Beregn antal unikke returkøb pr. Varenr.
return_counts = (
    df_returns
    .groupby('Varenr.')
    .size()
    .reset_index(name='Retur')
)

# 4d. Slå sammen og beregn nettokøbet
counts = (
    buy_counts
    .merge(return_counts, on='Varenr.', how='left')
    .fillna(0)
)
counts['Antal_køb'] = (counts['Køb'] - counts['Retur']).astype(int)
counts = counts[['Varenr.', 'Antal_køb']]

# 5. Slå Antal_køb ind i det eksisterende dataframe
df_merged = df_sales.merge(counts, on='Varenr.', how='left')
df_merged['Antal_køb'] = df_merged['Antal_køb'].fillna(0).astype(int)

# 6. Indsæt kolonnen lige efter 'Antal_salg'
idx = df_merged.columns.get_loc('Antal_salg')
df_merged.insert(idx + 1, 'Antal_køb', df_merged.pop('Antal_køb'))

# 7. Gem til ny Excel-fil
df_merged.to_excel(output_path, index=False)

print("Færdig! Fil gemt som:", output_path)


Færdig! Fil gemt som: vareposter_antal_salg_køb_modregning.xlsx


In [21]:
import pandas as pd

# Indlæs data
df_antal = pd.read_excel('antal_per_varenr_leverandor.xlsx')
df_vareposter = pd.read_excel('vareposter_antal_salg_køb_modregning.xlsx')

# Opret mapping fra Varenr. til absolutte Antal
mapping = df_antal.set_index('Varenr.')['Antal'].abs()

# Opdater kolonnen Solgt_per_leverandør
df_vareposter['Solgt_per_leverandør'] = (
    df_vareposter['Varenr.']
    .map(mapping)
    .fillna(df_vareposter['Solgt_per_leverandør'])
)

# Gem den opdaterede fil
df_vareposter.to_excel(
    'vareposter_antal_salg_køb_modregning_opdateret.xlsx',
    index=False
)

print("Opdatering færdig. Fil gemt som 'vareposter_antal_salg_køb_modregning_opdateret.xlsx'")

Opdatering færdig. Fil gemt som 'vareposter_antal_salg_køb_modregning_opdateret.xlsx'


In [28]:
import pandas as pd

# Indlæs det opdaterede primære datasæt
df_vareposter = pd.read_excel('vareposter_antal_salg_køb_modregning_opdateret.xlsx')

# Indlæs datasættet med købstal
df_purchased = pd.read_excel('antal_per_varenr_leverandor_purchased.xlsx')

# Opret mapping fra Varenr. til absolutte Antal (Købt)
mapping_purchased = df_purchased.set_index('Varenr.')['Antal'].abs()

# Find positionen lige til højre for "Solgt_per_leverandør"
cols = list(df_vareposter.columns)
pos = cols.index('Solgt_per_leverandør') + 1

# Indsæt kolonnen "Købt_per_leverandør"
df_vareposter.insert(
    loc=pos,
    column='Købt_per_leverandør',
    value=df_vareposter['Varenr.'].map(mapping_purchased)
)

# Gem det nye datasæt
df_vareposter.to_excel(
    'vareposter_antal_salg_køb_modregning_opdateret_med_købt.xlsx',
    index=False
)

print("Kolonnen 'Købt_per_leverandør' er tilføjet og fil gemt som 'vareposter_antal_salg_køb_modregning_opdateret_med_købt.xlsx'.")


Kolonnen 'Købt_per_leverandør' er tilføjet og fil gemt som 'vareposter_antal_salg_køb_modregning_opdateret_med_købt.xlsx'.
